In [1]:
import pandas as pd
import numpy as np
import itertools
import datetime
import matplotlib.pyplot as plt
from PIL import Image
from datetime import *
from datetime import datetime, timedelta, date
# %load_ext google.colab.data_table
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
project_id = "perceptive-ivy-290216"

# Standard plotly imports
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
import fastf1
session = fastf1.get_session(2024, 'Austrian Grand Prix', 'R')
session.load()

circuit_info = session.get_circuit_info()

req         WARNING 	DEFAULT CACHE ENABLED! (1.69 GB) /Users/rdesh723/Library/Caches/fastf1
core           INFO 	Loading data for Austrian Grand Prix - Race [v3.3.8]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['63', '81', '55', '44', '1', '27', '11', '20', '3', '10', '16', '31', '18', '22', '23

In [4]:
query2=f"""
SELECT
A.Year,
A.GP,
A.Driver,
A.DriverNumber,
A.Team,
CAST(A.Position AS INT64) Position,
Status,
A.LapStartTime,
A.Time LapEndTime,
B.Time as LapInstant,
A.LapTime,
CAST(LapNumber as INT64) LapNumber,
RPM,
Speed,
nGear,
Throttle,
Brake,
DRS,
Distance,
RelativeDistance,
DriverAhead,
DistanceToDriverAhead,
X,
Y,
Z,
CAST(Stint as INT64) Stint,
PitOutTime,
PitInTime,
Sector1Time,
Sector2Time,
Sector3Time,
Sector1SessionTime,
Sector2SessionTime,
Sector3SessionTime,
CAST(SpeedI1 AS INT64) SpeedI1,
CAST(SpeedI2 AS INT64) SpeedI2,
CAST(SpeedFL AS INT64) SpeedFL,
CAST(SpeedST AS INT64) SpeedST,
Compound,
CAST(TyreLife AS INT64) TyreLife,
FreshTyre,
TrackStatus,
Deleted,
DeletedReason
FROM `perceptive-ivy-290216.f1_api.qualifying_lap_time`  A
INNER JOIN
`perceptive-ivy-290216.f1_api.qualifying_telemetry_2024` B
ON B.SessionTime>=A.LapStartTime AND B.SessionTime<=A.Time
AND A.Driver=B.Driver
AND A.Year=B.Year
AND A.GP=B.GP
WHERE A.Year=2024
AND A.GP="Austrian Grand Prix"
# AND A.DRIVER='HAM'
AND B.Year=2024
AND B.GP="Austrian Grand Prix"
# AND B.Driver='HAM'
ORDER BY LapNumber, LapStartTime, LapInstant
"""
track1=pd.read_gbq(query2,project_id,dialect='standard')

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_3956/24620322.py:62: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  track1=pd.read_gbq(query2,project_id,dialect='standard')


In [5]:
track2=track1.copy()
year=track2['Year'][0]
gp=track2['GP'][0]

,Year,GP,Driver,DriverNumber,Team,Position,Status,LapStartTime,LapEndTime,LapInstant,LapTime,LapNumber,RPM,Speed,nGear,Throttle,Brake,DRS,Distance,RelativeDistance,DriverAhead,DistanceToDriverAhead,X,Y,Z,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,Sector3Time,Sector1SessionTime,Sector2SessionTime,Sector3SessionTime,SpeedI1,SpeedI2,SpeedFL,SpeedST,Compound,TyreLife,FreshTyre,TrackStatus,Deleted,DeletedReason
86178,2024,Austrian Grand Prix,HAM,44,Mercedes,<NA>,OnTrack,0 days 00:29:02.237000,0 days 00:30:07.778000,0 days 00:10:51.113000,0 days 00:01:05.541000,5,11305,291,7,100,False,12,16849.430000,0.220138,2,448.990556,987.792847,-1238.460788,7213.998771,2,NaT,NaT,0 days 00:00:16.543000,0 days 00:00:29.188000,0 days 00:00:19.810000,0 days 00:29:18.780000,0 days 00:29:47.968000,0 days 00:30:07.778000,319,245,284,318,SOFT,2,True,1,False,
86179,2024,Austrian Grand Prix,HAM,44,Mercedes,<NA>,OnTrack,0 days 00:29:02.237000,0 days 00:30:07.778000,0 days 00:10:51.131000,0 days 00:01:05.541000,5,11055,291,7,100,False,12,16850.886327,0.220157,2,442.679444,975.000000,-1242.000000,7214.000000,2,NaT,NaT,0 days 00:00:16.543000,0 days 00:00:29.188000,0 days 00:00:19.810000,0 days 00:29:18.780000,0 days 00:29:47.968000,0 days 00:30:07.778000,319,245,284,318,SOFT,2,True,1,False,
86180,2024,Austrian Grand Prix,HAM,44,Mercedes,<NA>,OnTrack,0 days 00:29:02.237000,0 days 00:30:07.778000,0 days 00:10:51.311000,0 days 00:01:05.541000,5,10805,291,7,100,False,12,16865.465612,0.220348,2,436.368333,834.000000,-1281.000000,7214.000000,2,NaT,NaT,0 days 00:00:16.543000,0 days 00:00:29.188000,0 days 00:00:19.810000,0 days 00:29:18.780000,0 days 00:29:47.968000,0 days 00:30:07.778000,319,245,284,318,SOFT,2,True,1,False,
86181,2024,Austrian Grand Prix,HAM,44,Mercedes,<NA>,OnTrack,0 days 00:29:02.237000,0 days 00:30:07.778000,0 days 00:10:51.353000,0 days 00:01:05.541000,5,10556,292,8,100,False,12,16868.896667,0.220393,2,430.057222,801.436865,-1289.768023,7213.999641,2,NaT,NaT,0 days 00:00:16.543000,0 days 00:00:29.188000,0 days 00:00:19.810000,0 days 00:29:18.780000,0 days 00:29:47.968000,0 days 00:30:07.778000,319,245,284,318,SOFT,2,True,1,False,
86182,2024,Austrian Grand Prix,HAM,44,Mercedes,<NA>,OnTrack,0 days 00:29:02.237000,0 days 00:30:07.778000,0 days 00:10:51.511000,0 days 00:01:05.541000,5,10568,295,8,100,False,12,16881.941882,0.220563,2,415.507222,676.000000,-1323.000000,7214.000000,2,NaT,NaT,0 days 00:00:16.543000,0 days 00:00:29.188000,0 days 00:00:19.810000,0 days 00:29:18.780000,0 days 00:29:47.968000,0 days 00:30:07.778000,319,245,284,318,SOFT,2,True,1,False,


In [6]:
#Convert braking from True False to 0 & 1
def braking_ind(ind):
  if ind==True:
    return 1
  else:
    return 0

track2['Braking'] = track2['Brake'].apply(braking_ind)

In [7]:
track2["LapTime"]=track2['LapTime'].str.split('days ').str[1]
track2["LapTime"]=pd.to_datetime(track2['LapTime'],format='%H:%M:%S.%f')
track2["LapTime"]=track2["LapTime"].dt.strftime("%H:%M:%S.%f")

In [8]:
#Convert lapnumber to string for discrete chart coloring
track2["LapNumber"]=track2["LapNumber"].astype(str)

In [9]:
track2['Dist_Diff2'] = track2.groupby(["LapNumber","Driver"])["Distance"].shift(1)
# track2['Dist_Diff2'] = track2.groupby(["Driver"])["Distance"].shift(1) ## Use when not looking at just 1 lap
track2['Dist_Diff'] = track2['Distance']-track2["Dist_Diff2"]
track2["Dist_Diff_Sum"] = track2.groupby(by=["LapNumber","Driver"])["Dist_Diff"].cumsum()
# track2["Dist_Diff_Sum"] = track2.groupby(by=["Driver"])["Dist_Diff"].cumsum()  ## Use when not looking at just 1 lap

##Another way to calculate distances manually, if the distance column wasnt present in the data set. Dist_Diff_Sum AND Dist_Total Should match very closely which they do, even when using our own method

#Calculate the distance traversed in totality and then for each lap
track2['X_Diff'] = track2['X'] - track2['X'].shift(1)
track2['Y_Diff'] = track2['Y'] - track2['Y'].shift(1)
track2['Z_Diff'] = track2['Z'] - track2['Z'].shift(1)
track2['X_Diff']=track2['X_Diff'].pow(2)
track2['Y_Diff']=track2['Y_Diff'].pow(2)
track2['Z_Diff']=track2['Z_Diff'].pow(2)
track2['Dist']=track2['X_Diff']+track2['Y_Diff']+track2['Z_Diff']

#Distance between current point and previous data point
track2['Dist']=track2['Dist'].pow(1./2)
track2['Dist']=track2['Dist']/10

#Running sum of distance covered by a driver
track2['Dist_Total']=track2['Dist'].cumsum()

#Running sum of distance covered by a driver by lap
track2["Dist_Lap"] = track2.groupby(by=["LapNumber","Driver"])["Dist"].cumsum()

# #Drop columns that are not necessary anymore
# track2.drop(['X_Diff','Y_Diff','Z_Diff','Dist_Diff2'], axis=1, inplace=True)

# #Assign Rank for each entry point
track2["RK"] = track2.groupby(by=["LapNumber","Driver"])["LapInstant"].rank(method="dense", ascending=True)
track2["RK"]= track2["RK"].astype(int)
track2=track2.sort_values(by=['Driver','LapInstant'])

In [10]:
# track2[(track2["Driver"]=='HAM')&((track2["LapNumber"]=='1')|(track2["LapNumber"]=='2'))].sort_values(by=["LapNumber","LapInstant"])

In [11]:
# pd.DataFrame(track2.groupby(by=["Driver"])["LapTime"].unique())
Lap_Driver=pd.DataFrame(track2.groupby(by=["Driver","LapNumber"])["LapTime"].unique().explode())
Lap_Driver=Lap_Driver.sort_values(by=["Driver","LapTime"])
Lap_Driver["RK"] = Lap_Driver.groupby("Driver")["LapTime"].rank(method="dense", ascending=True)
Lap_Driver.sort_values(by=["LapTime"]).head()

LapTime   RK
Driver LapNumber                      
VER    17         00:01:04.314000  1.0
       14         00:01:04.426000  2.0
       11         00:01:04.469000  3.0
       8          00:01:04.577000  4.0
NOR    19         00:01:04.718000  1.0

In [12]:
#Add rank for each lap in the main table to plot easily
track2["LapRank"] = track2.groupby("Driver")["LapTime"].rank(method="dense", ascending=True)

In [13]:
fig_speed=px.line(data_frame=track2[(track2['Driver'].isin(['HAM','VER'])&(track2['LapRank']==1.0))], x="Dist_Diff_Sum", y="Speed",
           color="Driver", hover_name="Driver",
           height=900, width=1800,
           template="plotly_dark",
           title="Speed Trace For GP",
           color_discrete_map={
                 "ALB": "#64C4FF",
                 "ALO": "#229971",
                 "BOT": "#52e252",
                 "GAS": "#0093cc",
                 "HAM": "#27F4D2",
                 "HUL": "#B6BABD",
                 "LEC": "#E80020",
                 "MAG": "#B6BABD",
                 "NOR": "#FF8000",
                 "OCO": "#0093cc",
                 "PER": "#3671C6",
                 "PIA": "#FF8000",
                 "RIC": "#6692FF",
                 "RUS": "#27F4D2",
                 "SAI": "#E80020",
                 "SAR": "#37BEDD",
                 "STR": "#229971",
                 "TSU": "#6692FF",
                 "VER": "#3671C6",
                 "ZHO": "#52e252"
          }
          )
# fig_speed.add_layout_image(
#     dict(
#         source=Image.open("/content/images.jpg"),
#         xref="x", yref="y",
#         x=4500, y=300,
#         sizex=500, sizey=500,
#         xanchor="left", yanchor="middle"
#     )
# )
fig_speed.update_layout(
    xaxis_title="Distance (m)",
    yaxis_title="Speed (Km/Hr)",
)

fig_speed.update_layout(
    title_x=0.5,
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=15)),
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
    ),
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="White"
    ),
    title_font_family="PT Sans Narrow"
#  "Arial", "Balto", "Courier New", "Droid Sans", "Droid Serif", "Droid Sans Mono", "Gravitas One", "Old Standard TT", "Open Sans", "Overpass", "PT Sans Narrow", "Raleway", "Times New Roman"
)
fig_speed.show()

In [14]:
fig_speed_addtl=px.line(track2[(track2['Driver'].isin(['LEC'])&(track2['LapRank']==1.0))], x="Dist_Diff_Sum", y="Speed",
           color="Driver", hover_name="Driver",
           height=900, width=1800,
           template="plotly_dark",
           title="Speed Trace For GP",
           color_discrete_map={
                 "ALB": "#64C4FF",
                 "ALO": "#229971",
                 "BOT": "#52e252",
                 "GAS": "#0093cc",
                 "HAM": "#27F4D2",
                 "HUL": "#B6BABD",
                 "LEC": "#E80020",
                 "MAG": "#B6BABD",
                 "NOR": "#FF8000",
                 "OCO": "#0093cc",
                 "PER": "#3671C6",
                 "PIA": "#FF8000",
                 "RIC": "#6692FF",
                 "RUS": "#27F4D2",
                 "SAI": "#E80020",
                 "SAR": "#37BEDD",
                 "STR": "#229971",
                 "TSU": "#6692FF",
                 "VER": "#3671C6",
                 "ZHO": "#52e252"
          }
          )

fig_speed.add_traces(
    list(fig_speed_addtl.select_traces())
)

fig_speed.update_layout(
    xaxis_title="Distance (m)",
    yaxis_title="Speed (Km/Hr)",
)
fig_speed.update_layout(
    title_x=0.5,
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
    ),
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=15)),
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="White"
    ),
    title_font_family="PT Sans Narrow"
#  "Arial", "Balto", "Courier New", "Droid Sans", "Droid Serif", "Droid Sans Mono", "Gravitas One", "Old Standard TT", "Open Sans", "Overpass", "PT Sans Narrow", "Raleway", "Times New Roman"
)

fig_speed.show()

In [15]:
#Speed Trace with Corner Markings

In [16]:
v_min = track2['Speed'].min()
v_max = track2['Speed'].max()
corner_distance=circuit_info.corners['Distance']

for k in corner_distance:
  fig_speed.add_shape(type='line',
                yref="y",
                xref="x",
                x0=k,
                y0=v_min+25,
                x1=k,
                y1=v_max+20,
                line_width=1,
                line_dash="dash",
                line_color="grey")
fig_speed.update_xaxes(range=[0, 5000])
fig_speed.update_yaxes(range=[10, 350])

for _, corner in circuit_info.corners.iterrows():
    txt = f"{corner['Number']}{corner['Letter']}"
    fig_speed.add_annotation(x=corner['Distance'], y=v_min+15, text=txt, showarrow=False, yshift=10)

fig_speed.update_layout(
   title_x=0.5,
   hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
    ),
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=15)),
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="White"
    ),
    title_font_family="PT Sans Narrow"
#  "Arial", "Balto", "Courier New", "Droid Sans", "Droid Serif", "Droid Sans Mono", "Gravitas One", "Old Standard TT", "Open Sans", "Overpass", "PT Sans Narrow", "Raleway", "Times New Roman"
)

fig_speed.show()

In [17]:
fig_throttle=px.line(data_frame=track2[(track2['Driver'].isin(['HAM','VER','LEC'])&(track2['LapRank']==1.0))], x="Dist_Diff_Sum", y="Throttle",
           color="Driver", hover_name="Driver",
           height=900, width=1800,
           template="plotly_dark",
           title="Throttle Trace For GP",
           color_discrete_map={
                 "ALB": "#64C4FF",
                 "ALO": "#229971",
                 "BOT": "#52e252",
                 "GAS": "#0093cc",
                 "HAM": "#27F4D2",
                 "HUL": "#B6BABD",
                 "LEC": "#E80020",
                 "MAG": "#B6BABD",
                 "NOR": "#FF8000",
                 "OCO": "#0093cc",
                 "PER": "#3671C6",
                 "PIA": "#FF8000",
                 "RIC": "#6692FF",
                 "RUS": "#27F4D2",
                 "SAI": "#E80020",
                 "SAR": "#37BEDD",
                 "STR": "#229971",
                 "TSU": "#6692FF",
                 "VER": "#3671C6",
                 "ZHO": "#52e252"
          }
          )

fig_throttle.update_layout(
    xaxis_title="Distance (m)",
    yaxis_title="DRS",
)

v_min = track2['Throttle'].min()
v_max = track2['Throttle'].max()
corner_distance=circuit_info.corners['Distance']

for k in corner_distance:
  fig_throttle.add_shape(type='line',
                yref="y",
                xref="x",
                x0=k,
                y0=v_min-5,
                x1=k,
                y1=v_max+20,
                line_width=1,
                line_dash="dash",
                line_color="grey")
fig_throttle.update_xaxes(range=[0, 5000])
fig_throttle.update_yaxes(range=[-10, 110])

for _, corner in circuit_info.corners.iterrows():
    txt = f"{corner['Number']}{corner['Letter']}"
    fig_throttle.add_annotation(x=corner['Distance'], y=v_min-7, text=txt, showarrow=False, yshift=10)

fig_throttle.update_layout(
   title_x=0.5,
   hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
    ),
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=15)),
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="White"
    ),
    title_font_family="PT Sans Narrow"
#  "Arial", "Balto", "Courier New", "Droid Sans", "Droid Serif", "Droid Sans Mono", "Gravitas One", "Old Standard TT", "Open Sans", "Overpass", "PT Sans Narrow", "Raleway", "Times New Roman"
)

fig_throttle.show()

In [18]:
fig_brake=px.line(data_frame=track2[(track2['Driver'].isin(['HAM','VER','LEC'])&(track2['LapRank']==1.0))], x="Dist_Diff_Sum", y="Braking",
           color="Driver", hover_name="Driver",
           height=900, width=1800,
           template="plotly_dark",
           title="Brake Trace For GP",
           color_discrete_map={
                 "ALB": "#64C4FF",
                 "ALO": "#229971",
                 "BOT": "#52e252",
                 "GAS": "#0093cc",
                 "HAM": "#27F4D2",
                 "HUL": "#B6BABD",
                 "LEC": "#E80020",
                 "MAG": "#B6BABD",
                 "NOR": "#FF8000",
                 "OCO": "#0093cc",
                 "PER": "#3671C6",
                 "PIA": "#FF8000",
                 "RIC": "#6692FF",
                 "RUS": "#27F4D2",
                 "SAI": "#E80020",
                 "SAR": "#37BEDD",
                 "STR": "#229971",
                 "TSU": "#6692FF",
                 "VER": "#3671C6",
                 "ZHO": "#52e252"
          }
          )

fig_brake.update_layout(
    xaxis_title="Distance (m)",
    yaxis_title="Brakes",
)

v_min = track2['Braking'].min()
v_max = track2['Braking'].max()
corner_distance=circuit_info.corners['Distance']

for k in corner_distance:
  fig_brake.add_shape(type='line',
                yref="y",
                xref="x",
                x0=k,
                y0=v_min-0.25,
                x1=k,
                y1=v_max+0.2,
                line_width=1,
                line_dash="dash",
                line_color="grey")
fig_brake.update_xaxes(range=[0, 5000])
fig_brake.update_yaxes(range=[-0.5, 1.2])

for _, corner in circuit_info.corners.iterrows():
    txt = f"{corner['Number']}{corner['Letter']}"
    fig_brake.add_annotation(x=corner['Distance'], y=v_min-0.3, text=txt, showarrow=False, yshift=10)

fig_brake.update_layout(
   title_x=0.5,
   hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
    ),
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=15)),
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="White"
    ),
    title_font_family="PT Sans Narrow"
#  "Arial", "Balto", "Courier New", "Droid Sans", "Droid Serif", "Droid Sans Mono", "Gravitas One", "Old Standard TT", "Open Sans", "Overpass", "PT Sans Narrow", "Raleway", "Times New Roman"
)

fig_brake.show()

In [19]:
fig_drs=px.line(data_frame=track2[(track2['Driver'].isin(['HAM','VER','LEC'])&(track2['LapRank']==1.0))], x="Dist_Diff_Sum", y="DRS",
           color="Driver", hover_name="Driver",
           height=900, width=1800,
           template="plotly_dark",
           title="DRS Trace For GP",
           color_discrete_map={
                 "ALB": "#64C4FF",
                 "ALO": "#229971",
                 "BOT": "#52e252",
                 "GAS": "#0093cc",
                 "HAM": "#27F4D2",
                 "HUL": "#B6BABD",
                 "LEC": "#E80020",
                 "MAG": "#B6BABD",
                 "NOR": "#FF8000",
                 "OCO": "#0093cc",
                 "PER": "#3671C6",
                 "PIA": "#FF8000",
                 "RIC": "#6692FF",
                 "RUS": "#27F4D2",
                 "SAI": "#E80020",
                 "SAR": "#37BEDD",
                 "STR": "#229971",
                 "TSU": "#6692FF",
                 "VER": "#3671C6",
                 "ZHO": "#52e252"
          }
          )

fig_drs.update_layout(
    xaxis_title="Distance (m)",
    yaxis_title="DRS",
)

v_min = track2['DRS'].min()
v_max = track2['DRS'].max()
corner_distance=circuit_info.corners['Distance']

for k in corner_distance:
  fig_drs.add_shape(type='line',
                yref="y",
                xref="x",
                x0=k,
                y0=v_min-5,
                x1=k,
                y1=v_max+2,
                line_width=1,
                line_dash="dash",
                line_color="grey")
fig_drs.update_xaxes(range=[0, 5000])
fig_drs.update_yaxes(range=[-0.5, 18])

for _, corner in circuit_info.corners.iterrows():
    txt = f"{corner['Number']}{corner['Letter']}"
    fig_drs.add_annotation(x=corner['Distance'], y=v_min-5.5, text=txt, showarrow=False, yshift=10)
fig_drs.update_layout(
   title_x=0.5,
   hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
    ),
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=15)),
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="White"
    ),
    title_font_family="PT Sans Narrow"
#  "Arial", "Balto", "Courier New", "Droid Sans", "Droid Serif", "Droid Sans Mono", "Gravitas One", "Old Standard TT", "Open Sans", "Overpass", "PT Sans Narrow", "Raleway", "Times New Roman"
)
fig_drs.show()

In [20]:
#Plot as a subplot
import plotly.express as px
import plotly.subplots as sp


# For as many traces that exist per Express figure, get the
# traces from each plot and store them in an array.
# This is essentially breaking down the Express fig into it's traces
figure1_traces = []
figure2_traces = []
figure3_traces = []
figure4_traces = []

for trace in range(len(fig_speed["data"])):
    figure1_traces.append(fig_speed["data"][trace])
for trace in range(len(fig_throttle["data"])):
    figure2_traces.append(fig_throttle["data"][trace])
for trace in range(len(fig_brake["data"])):
    figure3_traces.append(fig_brake["data"][trace])
for trace in range(len(fig_drs["data"])):
    figure4_traces.append(fig_drs["data"][trace])

this_figure = sp.make_subplots(rows=4, cols=1)

for traces in figure1_traces:
    this_figure.append_trace(traces, row=1, col=1)
for traces in figure2_traces:
    this_figure.append_trace(traces, row=2, col=1)
for traces in figure3_traces:
    this_figure.append_trace(traces, row=3, col=1)
for traces in figure4_traces:
    this_figure.append_trace(traces, row=4, col=1)

this_figure.update_layout(height=1800, width=1800, title_text="Lap Traces")

this_figure.update_layout(
    title_x=0.5,
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=15)),
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
    ),
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="Black"
    ),
    title_font_family="PT Sans Narrow"
#  "Arial", "Balto", "Courier New", "Droid Sans", "Droid Serif", "Droid Sans Mono", "Gravitas One", "Old Standard TT", "Open Sans", "Overpass", "PT Sans Narrow", "Raleway", "Times New Roman"
)
this_figure.show()